# Srock Strategy - Factor Strategy 0004 - Factor-Based Stock Selection 

## 1. Import Library

In [1]:
import dai
import random
import pandas as pd

## 2. Prepare Strategy Data

In [2]:
sd = '2022-12-31'
ed = '2026-02-13'

In [3]:
sql_factor =  f"""
SELECT 
    date, 
    instrument,
    -1 * float_market_cap AS factor
FROM cn_stock_factors
WHERE st_status = 0
AND suspended = 0
AND trading_days > 270
AND list_sector = 1
AND price_limit_status = 2
QUALIFY COLUMNS(*) IS NOT NULL
AND c_pct_rank(dividend_yield_ratio) BETWEEN 0.75 AND 1.00
AND close < 10
ORDER BY date, float_market_cap DESC
"""

In [4]:
sql_trade = f"""
WITH
data_strategy AS (
    {sql_factor}
),
data_filter AS (
    SELECT
        date,
        instrument,
        factor AS score,
        c_rank(factor) AS score_rank,
    FROM data_strategy
    QUALIFY score_rank <= 5
),
data_date AS (
    SELECT
        date,
        instrument,
        score, 
        score_rank, 
        1 / c_sum(1) AS position, 
    FROM data_filter JOIN mldt_cn_stock_calendar_daily USING (date)
    WHERE 1=1
)
SELECT *
FROM data_date
ORDER BY date, score_rank
"""

In [5]:
df_strategy = dai.query(sql_trade, filters = {"date":[sd, ed]}).df()
df_strategy

,date,instrument,score,score_rank,position
0,2023-01-03,601398.SH,-1.162029e+12,1.0,0.2
1,2023-01-03,601288.SH,-8.679252e+11,2.0,0.2
2,2023-01-03,601857.SH,-8.112296e+11,3.0,0.2
3,2023-01-03,601988.SH,-6.660190e+11,4.0,0.2
4,2023-01-03,601319.SH,-1.877831e+11,5.0,0.2
...,...,...,...,...,...
3780,2026-02-13,601728.SH,-4.401606e+11,1.0,0.2
3781,2026-02-13,601658.SH,-3.403105e+11,2.0,0.2
3782,2026-02-13,601816.SH,-2.402251e+11,3.0,0.2
3783,2026-02-13,601766.SH,-1.535084e+11,4.0,0.2


## 3. Backtest Strategy

In [6]:
from bigmodule import M

def BigTrader_Initialize(context):
    from bigtrader.finance.commission import PerOrder
    context.set_commission(PerOrder(buy_cost=0.0003, sell_cost=0.0013, min_cost=5))

def BigTrader_Before_Trading(context, data):
    pass

def BigTrader_Handle_Tick(context, tick):
    pass

def BigTrader_Handle_Data(context, data):

    df_now = context.data[context.data["date"] == data.current_dt.strftime("%Y-%m-%d")]

    if len(df_now) == 0:
        return
    
    target_instruments  = list(df_now["instrument"])
    holding_instruments = list(context.get_account_positions().keys())

    for instrument in holding_instruments:
        if instrument not in target_instruments:
            context.order_target_percent(instrument, 0)
        
    for i, x in df_now.iterrows():
        position = 0.0 if pd.isnull(x.position) else float(x.position)
        context.order_target_percent(x.instrument, position)

def BigTrader_Handle_Trade(context, trade):
    pass

def BigTrader_Handle_Order(context, order):
    pass

def BigTrader_After_Trading(context, data):
    pass

BigTrader = M.bigtrader.v34(
    
    data = df_strategy,
    
    start_date = """""",
    end_date   = """""",
    
    initialize           = BigTrader_Initialize,
    before_trading_start = BigTrader_Before_Trading,
    handle_tick          = BigTrader_Handle_Tick,
    handle_data          = BigTrader_Handle_Data,
    handle_trade         = BigTrader_Handle_Trade,
    handle_order         = BigTrader_Handle_Order,
    after_trading        = BigTrader_After_Trading,
    
    capital_base = 500000  + random.uniform(0, 10),
    frequency="""daily""",
    product_type="""自动""",
    rebalance_period_type="""交易日""",
    rebalance_period_days="""1""",
    rebalance_period_roll_forward=True,
    backtest_engine_mode="""标准模式""",
    before_start_days=0,
    volume_limit=1,
    order_price_field_buy="""close""",
    order_price_field_sell="""close""",
    benchmark="""沪深300指数""",
    
    plot_charts=True,
    debug=False,
    backtest_only=False,
    m_name="""BigTrader"""
) 

[2026-02-18 19:29:21] [info     ] bigtrader.v34 开始运行 ..
[2026-02-18 19:29:21] [info     ] 2023-01-03, 2026-02-13, , , instruments=22
[2026-02-18 19:29:21] [info     ] bigtrader module V2.2.0
[2026-02-18 19:29:21] [info     ] bigtrader engine v0.1.0.post9+g7a244b6 2026-02-10
[2026-02-18 19:29:25] [info     ] backtest done, raw_perf_ds:dai.DataSource("_6ba631d46e7445cc876d50d08d0ad2e2")


[2026-02-18 19:29:26] [info     ] bigtrader.v34 运行完成 [5.015s].
